In [8]:
%load_ext autoreload
%autoreload 2
import yaml
import pathlib
from os.path import join
import os
import sys
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

current_path = str(pathlib.Path(os.getcwd()).parent)
sys.path.append(current_path)

#import sys
#sys.path.append('../)

from config_utils import *
from sequtils import *
from training_utils import *
from prokbert_tokenizer import ProkBERTTokenizer
from prok_datasets import IterableProkBERTPretrainingDataset
from general_utils import *
from ProkBERTDataCollator import *
from transformers import MegatronBertConfig, MegatronBertForMaskedLM, Trainer, TrainingArguments

    
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 30)
#pd.set_option('display.width', 4000)
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', True)
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2

train_model_output_dir = ''
train_model_name = 'LCA_repr_k6s1_l2048hs1536_h16_l16_mlm02_rp8_rndp1_rkv'

model_params = {'model_outputpath': '/scratch/fastscratch/NBL/trained_models/test/mini0',
                'model_name' : 'mini0'}
dataset_params = {'dataset_path' : '../data/preprocessed/pretraining.h5'}
pretraining_params = {}


prokbert_config = ProkBERTConfig()
_ = prokbert_config.get_and_set_model_parameters(model_params)
_ = prokbert_config.get_and_set_dataset_parameters(dataset_params)
_ = prokbert_config.get_and_set_pretraining_parameters(pretraining_params)


# Kell egy dataset
hdf_file = '../data/preprocessed/pretraining.h5'

ds = IterableProkBERTPretrainingDataset(hdf_file, input_batch_size=10000, max_iteration_over_ds=5)

X = ds[0:3][:,0:10]
#torch.stack(ds[0:3])




2023-08-15 09:13:59,832 - INFO - Dataset size: 482


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
PRETRAIN_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/pretraining.yaml
SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
/home/ligeti/github/prokbert/src/prokbert
/home/ligeti/github/prokbert/src/prokbert


### Pretraining folyamata

Vannak előkészítő feladatok:
  * tokenizáló loadolása
  * collator osztály loadolása
  * Meglévő modellek ellenőrzése, checkpoint-ok ellenőrzése
  * dataset betöltése és inicializálása, ellenőrzése (megfelel-e a követelményeknek vagy sem)



In [9]:
nr_gpus = count_gpus()
 
tokenizer = get_training_tokenizer(prokbert_config)
prokbert_dc = get_data_collator_for_overlapping_sequences(tokenizer, prokbert_config)
hdf_file_exists, ds_size = check_hdf_dataset_file(prokbert_config)
[m_exists, cp_dir, cp, cps] = check_model_existance_and_checkpoint(prokbert_config.model_params['model_outputpath'], 
                                     prokbert_config.model_params['model_name'])
iteration_offset = get_the_iteration_offset(batch_size = prokbert_config.pretraining_params['per_device_train_batch_size'],
                                           training_steps = cps[-1],
                                           dataset_size = ds_size,
                                           nr_gpus =nr_gpus,
                                           radient_accumulation_steps = prokbert_config.pretraining_params['gradient_accumulation_steps'])
training_dataset = IterableProkBERTPretrainingDataset(file_path=prokbert_config.dataset_params['dataset_path'],
                                                     input_batch_size=prokbert_config.dataset_params['input_batch_size'],
                                                     ds_offset=iteration_offset)


if m_exists:
    print('Loading the existing model from the chekcpoint')
    expected_model_dir = cp_dir
    model = MegatronBertForMaskedLM.from_pretrained(expected_model_dir)
            
else:
    print('Investigating whether prevous model is exists')
    [init_m_exists, init_m_cp_dir, init_m_cp, init_m_cps] = check_model_existance_and_checkpoint(prokbert_config.model_params['model_outputpath'], 
                                     prokbert_config.model_params['model_name'])



# Get the training parameters:
training_args = TrainingArguments(
    output_dir=prokbert_config.pretraining_params.pop('output_dir'),  # Explicitly pass the 'output_dir'
    **prokbert_config.pretraining_params  # Pass the rest of the parameters
)


#prokbert_config.computation_params

2023-08-15 09:14:05,009 - INFO - Loading the datacollator class!
2023-08-15 09:14:05,009 - INFO - Collator Parameters:
  Number of tokens masked to left:  3
  Number of tokens masked to right: 2
  Probability of restoring a masked token: 0.8
  Probability of changing to a random token: 0.01
  MLM Probability: 0.05
  Default token type: torch.int16

2023-08-15 09:14:05,010 - INFO - Checking whether the file exists or not!
2023-08-15 09:14:05,010 - INFO - Loading and creating a IterableProkBERTPretrainingDataset
2023-08-15 09:14:05,011 - INFO - Dataset size: 482
2023-08-15 09:14:05,012 - INFO - model_path:  /scratch/fastscratch/NBL/trained_models/test/mini0
2023-08-15 09:14:05,013 - INFO -    The 0 is the largest checkpoint!
2023-08-15 09:14:05,014 - INFO - Dataset size: 482


SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
/home/ligeti/github/prokbert/src/prokbert
Loading the existing model from the chekcpoint


In [ ]:
prokbert_config.pretraining_params['adam_epsilon'].__class__


In [ ]:
prokbert_config.pretraining_params['adam_epsilon']


In [ ]:
prokbert_config.model_params

[m_exists, cp_dir, cp, cps] = check_model_existance_and_checkpoint(prokbert_config.model_params['model_outputpath'], 
                                     prokbert_config.model_params['model_name'])
# Conclusion: The current model extsts and can be resumed. 

# Cheking initiation model, if training required



In [ ]:
# checking hdf dataset
hdf_file_exists, ds_size = check_hdf_dataset_file(prokbert_config)





In [ ]:
prokbert_config.dataset_params


In [ ]:
print('Loading the datacollator class!')
prokbert_dc = ProkBERTDataCollator(tokenizer,
                                  mask_to_left=prokbert_config.data_collator_params['mask_to_left'], 
                                  mask_to_right=prokbert_config.data_collator_params['mask_to_right'],
                                  mlm_probability =   prokbert_config.data_collator_params['mlm_probability'],
                                  replace_prob =prokbert_config.data_collator_params['replace_prob'],
                                  random_prob = prokbert_config.data_collator_params['random_prob'])
print(str(prokbert_dc))

inputs, labels = prokbert_dc.torch_mask_tokens(ds[0:3][:,0:30])



In [ ]:
prokbert_config.default_torchtype